## Imports

In [2]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob

## Data Read-in

In [4]:
csv_files = glob.glob('*csv')

df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [31]:
BC_gf = gpd.read_file('countyboundary')

## Data Clean

In [6]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [7]:
df = df.dropna(subset=['SOLD DATE'])

In [9]:
# Define list of desired months (excluding current month)
desired_months = ['November']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [10]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    829
Name: count, dtype: int64
-------
$/SQUARE FEET
False    813
True      16
Name: count, dtype: int64
-------
YEAR BUILT
False    829
Name: count, dtype: int64
-------


In [46]:
# sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
# second_newest_building = sorted_df.iloc[2]
# print(second_newest_building['URL'])

In [11]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [12]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [13]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
59,PAST SALE,November-28-2023,Condo/Co-op,6301 N University Dr #217,Tamarac,FL,33321.0,200.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Tamarac/6301-N-Unive...,MARMLS,A11458751,N,Y,26.201422,-80.252998
108,PAST SALE,November-15-2023,Condo/Co-op,3970 Oaks Clubhouse Dr #109,Pompano Beach,FL,33069.0,320.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/3970-O...,Beaches MLS,F10385711,N,Y,26.223149,-80.179267
109,PAST SALE,November-16-2023,Condo/Co-op,650 NW 80th Ter #205,Margate,FL,33063.0,75000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/650-NW-80th-...,MARMLS,A11359305,N,Y,26.237544,-80.233068
117,PAST SALE,November-3-2023,Condo/Co-op,2271 NW 48th Ter #206,Lauderhill,FL,33313.0,75000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/2271-NW-4...,Beaches MLS,RX-10828637,N,Y,26.155164,-80.216402
113,PAST SALE,November-1-2023,Condo/Co-op,3301 Spanish Moss Ter #904,Lauderhill,FL,33319.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/3301-Span...,MARMLS,A11362016,N,Y,26.167267,-80.234440
92,PAST SALE,November-21-2023,Condo/Co-op,1500 NW 43 Ter #105,Lauderhill,FL,33313.0,85000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/1500-NW-4...,Beaches MLS,RX-10918048,N,Y,26.144822,-80.208122
83,PAST SALE,November-2-2023,Condo/Co-op,1500 NW 43rd Ter #203,Lauderhill,FL,33313.0,85000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderhill/1500-NW-4...,Beaches MLS,RX-10917613,N,Y,26.144822,-80.208122
106,PAST SALE,November-14-2023,Condo/Co-op,5100 SW 41st St #206,Pembroke Park,FL,33023.0,90000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Park/5100-S...,MARMLS,A11341495,N,Y,25.973443,-80.193348
81,PAST SALE,November-13-2023,Condo/Co-op,7561 NW 1st St #104,Margate,FL,33063.0,91000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/7561-NW-1st-...,Beaches MLS,F10401282,N,Y,26.232823,-80.224339
55,PAST SALE,November-14-2023,Condo/Co-op,9380 Sunrise Lakes Blvd #307,Sunrise,FL,33322.0,92000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/9380-Sunrise...,MARMLS,A11451850,N,Y,26.159303,-80.275154


In [19]:
print(df_filtered['URL'].iloc[59])

https://www.redfin.com/FL/Tamarac/6301-N-University-Dr-33321/unit-217/home/41558704


In [20]:
# Correct the prices, if needed
df_filtered.at[59,'PRICE']=(200000)
df_filtered.at[108,'PRICE']=(320000)

In [21]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [22]:
# # Corrections, if needed
df_filtered.at[59,'$/SQUARE FEET']=(200000/1040)
df_filtered.at[108,'$/SQUARE FEET']=(320000/1620)

In [23]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
117,75000.0,2271 NW 48th Ter #206,Lauderhill,81.0
54,95000.0,2801 NW 47th Ter Unit 201A,Lauderdale Lakes,93.0
113,80000.0,3301 Spanish Moss Ter #904,Lauderhill,94.0
81,91000.0,7561 NW 1st St #104,Margate,95.0
100,110000.0,3671 Environ Blvd #167,Lauderhill,100.0
64,95000.0,2271 NW 47th Ter #109,Lauderhill,102.0
109,75000.0,650 NW 80th Ter #205,Margate,102.0
56,106800.0,7605 NW 5th Pl #103,Margate,108.0
61,110000.0,7605 NW 5th Ct #107,Margate,111.0
92,85000.0,1500 NW 43 Ter #105,Lauderhill,112.0


In [19]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [25]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [26]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [28]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [32]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [36]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"November 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(BC_gf,tooltip='Broward County',name='Broward County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [37]:
m.save('index.html')

## Summary Info

In [38]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [39]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [40]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [41]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Pompano-Beach/3211-SE-11th-St-33062/unit-3E/home/41699734


In [42]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Four Seasons, 525 N Ft Lauderdale Bch Blvd #1903 Fort Lauderdale | Price $4,600,000 | $2,052 psf | Year built: 2022
Least Expensive
Castle Apts 8 Condo, 2271 NW 48th Ter #206 Lauderhill | Price $75,000 | $81 psf | Year built: 1973
Highest Price Per Square Foot
Four Seasons, 525 N Ft Lauderdale Bch Blvd #1402 Fort Lauderdale | Price $4,550,000 | $2,245 psf | Year built: 2022
Lowest Price Per Square Foot
Castle Apts 8 Condo, 2271 NW 48th Ter #206 Lauderhill | Price $75,000 | $81 psf | Year built: 1973
Newest
Solemar Beach Condo, 1116 N Ocean Blvd #502 Pompano Beach | Price $2,000,000 | $982 psf | Year built: 2023
Oldest
Pompano Surf Club Inc, 3211 SE 11 Unit 3e Pompano Beach | Price $660,000 | $528 psf | Year built: 1955


## Time on Market Calculator

In [84]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Sunrise/2000-Metropica-Way-33323/unit-2403/home/145208285


In [80]:
from datetime import datetime, timedelta

date1 = datetime(2023, 9, 2) ## List (Earlier) date
date2 = datetime(2023, 10, 10) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

38


## Map URL Snagger

In [75]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [76]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/BC_condo_sales_month_ending_oct_2023


## Get Summary Data

In [77]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 851
--------
Total sale price: $277,195,728
--------
Median sale price: $240,000
--------
Max sale price: $6,700,000
--------
Min sale price: $67,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,958
--------
Min price per square foot: $92
--------
Median price per square foot: $237
------------------------------------------------
CONDO AGES
Newest building: 2020.0
----------
Oldest building: 1955.0
----------
Average building age: 1980.4935370152762
